# Tskit-Simulation-実験

- https://tskit.dev/tsinfer/docs/stable/tutorial.html

## tskit 環境構築

In [1]:
!python --version

Python 3.8.16


In [2]:
!python3 -m pip install -q -r https://tskit.dev/tutorials/requirements.txt

     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 9.4 MB 10.9 MB/s 
     |████████████████████████████████| 7.2 MB 71.4 MB/s 
     |████████████████████████████████| 1.1 MB 50.3 MB/s 
     |████████████████████████████████| 5.8 MB 56.5 MB/s 
     |████████████████████████████████| 445 kB 50.1 MB/s 
     |████████████████████████████████| 15.2 MB 45.9 MB/s 
     |████████████████████████████████| 3.1 MB 72.6 MB/s 
     |████████████████████████████████| 3.3 MB 81.3 MB/s 
     |████████████████████████████████| 100 kB 9.7 MB/s 
     |████████████████████████████████| 121 kB 61.5 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 345 kB 82.6 MB/s 
     |████████████████████████████████| 41 kB 39 kB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 1.9 MB 75.6 MB/s 
     |███████████████████

In [3]:
!pip install -q tsinfer

     |████████████████████████████████| 276 kB 5.2 MB/s 
     |████████████████████████████████| 191 kB 74.8 MB/s 
     |████████████████████████████████| 6.7 MB 67.5 MB/s 


In [4]:
!pip install -q cyvcf2

     |████████████████████████████████| 6.8 MB 5.2 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 


In [5]:
import tskit, tsinfer, cyvcf2

print(f"tskit version:   {tskit.__version__}")   # 0.5.3   # Python 3.8.16
print(f"tsinfer version: {tsinfer.__version__}") # 0.3.0
print(f"cyvcf2 version:  {cyvcf2.__version__}")  # 0.30.18

tskit version:   0.5.3
tsinfer version: 0.3.0
cyvcf2 version:  0.30.18


## Simulation example

In [6]:
# サンプルファイルダウンロード

import builtins
import sys

import msprime
import tsinfer

if getattr(builtins, "__IPYTHON__", False):  # if running IPython: e.g. in a notebook
    from tqdm.notebook import tqdm
    num_diploids, seq_len = 100, 10_000
    name = "notebook-simulation"
else:  # Take parameters from the command-line
    from tqdm import tqdm
    num_diploids, seq_len = int(sys.argv[1]), float(sys.argv[2])
    name = "cli-simulation"
    
ts = msprime.sim_ancestry(
    num_diploids,
    population_size=10**4,
    recombination_rate=1e-8,
    sequence_length=seq_len,
    random_seed=6,
)
ts = msprime.sim_mutations(ts, rate=1e-8, random_seed=7)
ts.dump(name + "-source.trees")
print(
    f"Simulated {ts.num_samples} samples over {seq_len/1e6} Mb:",
    f"{ts.num_trees} trees and {ts.num_sites} sites"
)

with tsinfer.SampleData(
    path=name + ".samples",
    sequence_length=ts.sequence_length,
    num_flush_threads=2
) as sim_sample_data:
    for var in tqdm(ts.variants(), total=ts.num_sites):
        sim_sample_data.add_site(var.site.position, var.genotypes, var.alleles)
print("Stored output in", name + ".samples")

Simulated 200 samples over 0.01 Mb: 20 trees and 20 sites


  0%|          | 0/20 [00:00<?, ?it/s]

Stored output in notebook-simulation.samples


In [7]:
# サンプルデータ作成

tsinfer.SampleData.from_tree_sequence(ts, path="cli-simulation.samples", num_flush_threads=20)

In [8]:
# ファイルの確認
import glob
import os

files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f, os.stat(f).st_size)

cli-simulation.samples 53248
notebook-simulation.samples 53248
notebook-simulation-source.trees 36460


In [9]:
# サンプルデータ確認

!tsinfer ls cli-simulation.samples

path                  = cli-simulation.samples
file_size             = 52.0 KiB
format_name           = tsinfer-sample-data
format_version        = (5, 1)
finalised             = True
uuid                  = 7aba4d24-099b-4e80-9eb1-cacfea88c0dd
num_provenances       = 3
provenances/timestamp = shape=(3,); dtype=object;
provenances/record    = shape=(3,); dtype=object;
sequence_length       = 10000.0
metadata_schema       = {'codec': 'json'}
metadata              = {}
num_populations       = 1
num_individuals       = 100
num_samples           = 200
num_sites             = 20
populations/metadata_schema = {'additionalProperties': True, 'codec': 'json', 'properties': {'description': {'type': ['string', 'null']}, 'name': {'type': 'string'}}, 'required': ['name', 'description'], 'type': 'object'}
populations/metadata  = shape=(1,); dtype=object;
individuals/metadata_schema = None
individuals/metadata  = shape=(100,); dtype=object;
individuals/location  = shape=(100,); dtype=object;
individu

In [10]:
# treesファイル作成

tsinfer.infer(sim_sample_data).dump(name + ".trees")

In [11]:
# treesファイルのロード

ts = tskit.load("notebook-simulation.trees")

print(type(ts))
ts

<class 'tskit.trees.TreeSequence'>


In [12]:
# ファイルの確認
import glob
import os

files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f, os.stat(f).st_size)

cli-simulation.samples 53248
notebook-simulation.samples 53248
notebook-simulation.trees 24828
notebook-simulation-source.trees 36460


## 可視化

In [13]:
# 可視化
import IPython # for colab

subset = range(0, 10)  # show first 10 samples
limit = (0, 2_000)     # show only the trees covering the first 2kb
prefix = "notebook"    # Or use "cli" for the larger example

source_subset = ts.simplify(subset, filter_sites=False)
print(f"True tree seq, simplified to {len(subset)} sampled genomes")

svg_string = source_subset.draw_svg(size=(800, 200), x_lim=limit, time_scale="rank")

display(IPython.display.HTML(svg_string)) # for colab

True tree seq, simplified to 10 sampled genomes


## VCFファイルのロード

In [14]:
# Githubからファイルのダウンロード

!wget https://github.com/tskit-dev/tsinfer/raw/main/docs/_static/P_dom_chr24_phased.vcf.gz

--2022-12-25 08:18:13--  https://github.com/tskit-dev/tsinfer/raw/main/docs/_static/P_dom_chr24_phased.vcf.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tskit-dev/tsinfer/main/docs/_static/P_dom_chr24_phased.vcf.gz [following]
--2022-12-25 08:18:13--  https://raw.githubusercontent.com/tskit-dev/tsinfer/main/docs/_static/P_dom_chr24_phased.vcf.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127385 (124K) [application/octet-stream]
Saving to: ‘P_dom_chr24_phased.vcf.gz’

P_dom_chr24_phased. 100%[===================>] 124.40K  --.-KB/s    in 0.02s   

2022-12-25 08:18:13 (5.72 MB/s) - ‘P_do

In [15]:
# sample data

import cyvcf2
import tsinfer

def add_diploid_sites(vcf, samples):
    """
    Read the sites in the vcf and add them to the samples object.
    """
    # You may want to change the following line, e.g. here we allow
    # "*" (a spanning deletion) to be a valid allele state
    allele_chars = set("ATGCatgc*")
    pos = 0
    progressbar = tqdm(total=samples.sequence_length, desc="Read VCF", unit='bp')
    for variant in vcf:  # Loop over variants, each assumed at a unique site
        progressbar.update(variant.POS - pos)
        if pos == variant.POS:
            print(f"Duplicate entries at position {pos}, ignoring all but the first")
            continue
        else:
            pos = variant.POS
        if any([not phased for _, _, phased in variant.genotypes]):
            raise ValueError("Unphased genotypes for variant at position", pos)
        alleles = [variant.REF.upper()] + [v.upper() for v in variant.ALT]
        ancestral = variant.INFO.get("AA", ".")  # "." means unknown
        # some VCFs (e.g. from 1000G) have many values in the AA field: take the 1st
        ancestral = ancestral.split("|")[0].upper()
        if ancestral == "." or ancestral == "":
            ancestral_allele = MISSING_DATA
            # alternatively, you could specify `ancestral = variant.REF.upper()`
        else:
            ancestral_allele = alleles.index(ancestral)
        # Check we have ATCG alleles
        for a in alleles:
            if len(set(a) - allele_chars) > 0:
                print(f"Ignoring site at pos {pos}: allele {a} not in {allele_chars}")
                continue
        # Map original allele indexes to their indexes in the new alleles list.
        genotypes = [g for row in variant.genotypes for g in row[0:2]]
        samples.add_site(pos, genotypes, alleles, ancestral_allele=ancestral_allele)


def chromosome_length(vcf):
    assert len(vcf.seqlens) == 1
    return vcf.seqlens[0]


vcf_location = "P_dom_chr24_phased.vcf.gz"
# NB: could also read from an online version by setting vcf_location to
# "https://github.com/tskit-dev/tsinfer/raw/main/docs/_static/P_dom_chr24_phased.vcf.gz"

vcf = cyvcf2.VCF(vcf_location)
with tsinfer.SampleData(
    path="P_dom_chr24_phased.samples", sequence_length=chromosome_length(vcf)
) as samples:
    add_diploid_sites(vcf, samples)

print(
    "Sample file created for {} samples ".format(samples.num_samples)
    + "({} individuals) ".format(samples.num_individuals)
    + "with {} variable sites.".format(samples.num_sites),
    flush=True,
)

# Do the inference
ts = tsinfer.infer(samples)
print(
    "Inferred tree sequence: {} trees over {} Mb ({} edges)".format(
        ts.num_trees, ts.sequence_length / 1e6, ts.num_edges
    )
)

Read VCF:   0%|          | 0/7077728.0 [00:00<?, ?bp/s]

Sample file created for 20 samples (20 individuals) with 13192 variable sites.
Inferred tree sequence: 6845 trees over 7.077728 Mb (38045 edges)


In [16]:
# ファイルの確認
import glob
import os

files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f, os.stat(f).st_size)

P_dom_chr24_phased.vcf.gz 127385
cli-simulation.samples 53248
notebook-simulation.samples 53248
notebook-simulation.trees 24828
P_dom_chr24_phased.samples 245760
notebook-simulation-source.trees 36460


In [17]:
# 情報の追加

import json

def add_populations(vcf, samples):
    """
    Add tsinfer Population objects and returns a list of IDs corresponding to the VCF samples.
    """
    # In this VCF, the first letter of the sample name refers to the population
    samples_first_letter = [sample_name[0] for sample_name in vcf.samples]
    pop_lookup = {}
    pop_lookup["8"] = samples.add_population(metadata={"country": "Norway"})
    pop_lookup["F"] = samples.add_population(metadata={"country": "France"})
    return [pop_lookup[first_letter] for first_letter in samples_first_letter]


def add_diploid_individuals(vcf, samples, populations):
    for name, population in zip(vcf.samples, populations):
        samples.add_individual(ploidy=2, metadata={"name": name}, population=population)


# Repeat as previously but add both populations and individuals
vcf_location = "P_dom_chr24_phased.vcf.gz"
# NB: could also read from an online version by setting vcf_location to
# "https://github.com/tskit-dev/tsinfer/raw/main/docs/_static/P_dom_chr24_phased.vcf.gz"

vcf = cyvcf2.VCF(vcf_location)
with tsinfer.SampleData(
    path="P_dom_chr24_phased.samples", sequence_length=chromosome_length(vcf)
) as samples:
    populations = add_populations(vcf, samples)
    add_diploid_individuals(vcf, samples, populations)
    add_diploid_sites(vcf, samples)

print(
    "Sample file created for {} samples ".format(samples.num_samples)
    + "({} individuals) ".format(samples.num_individuals)
    + "with {} variable sites.".format(samples.num_sites),
    flush=True,
)

# Do the inference
sparrow_ts = tsinfer.infer(samples)
print(
    "Inferred tree sequence `{}`: {} trees over {} Mb".format(
        "sparrow_ts", sparrow_ts.num_trees, sparrow_ts.sequence_length / 1e6
    )
)
# Check the metadata
for sample_node_id in sparrow_ts.samples():
    individual_id = sparrow_ts.node(sample_node_id).individual
    population_id = sparrow_ts.node(sample_node_id).population
    print(
        "Node",
        sample_node_id,
        "labels a chr24 sampled from individual",
        json.loads(sparrow_ts.individual(individual_id).metadata),
        "in",
        json.loads(sparrow_ts.population(population_id).metadata)["country"],
    )


Read VCF:   0%|          | 0/7077728.0 [00:00<?, ?bp/s]

Sample file created for 20 samples (10 individuals) with 13192 variable sites.
Inferred tree sequence `sparrow_ts`: 6845 trees over 7.077728 Mb
Node 0 labels a chr24 sampled from individual {'name': '8934547'} in Norway
Node 1 labels a chr24 sampled from individual {'name': '8934547'} in Norway
Node 2 labels a chr24 sampled from individual {'name': '8L19766'} in Norway
Node 3 labels a chr24 sampled from individual {'name': '8L19766'} in Norway
Node 4 labels a chr24 sampled from individual {'name': '8M31651'} in Norway
Node 5 labels a chr24 sampled from individual {'name': '8M31651'} in Norway
Node 6 labels a chr24 sampled from individual {'name': '8N05890'} in Norway
Node 7 labels a chr24 sampled from individual {'name': '8N05890'} in Norway
Node 8 labels a chr24 sampled from individual {'name': '8N73604'} in Norway
Node 9 labels a chr24 sampled from individual {'name': '8N73604'} in Norway
Node 10 labels a chr24 sampled from individual {'name': 'FR041'} in France
Node 11 labels a chr2

## Analysis

In [18]:
colours = {"Norway": "red", "France": "blue"}
colours_for_node = {}
for n in sparrow_ts.samples():
    population_data = sparrow_ts.population(sparrow_ts.node(n).population)
    colours_for_node[n] = colours[json.loads(population_data.metadata)["country"]]

individual_for_node = {}
for n in sparrow_ts.samples():
    individual_data = sparrow_ts.individual(sparrow_ts.node(n).individual)
    individual_for_node[n] = json.loads(individual_data.metadata)["name"]

tree = sparrow_ts.at(1e6)
svg_string = tree.draw(
    path="tree_at_1Mb.svg",
    height=700,
    width=1200,
    node_labels=individual_for_node,
    node_colours=colours_for_node,
)

display(IPython.display.HTML(svg_string)) # for colab

### SFS

In [19]:
# site frequency spectrum (SFS)の計算

samples_listed_by_population = [
    sparrow_ts.samples(population=pop_id)
    for pop_id in range(sparrow_ts.num_populations)
]

print("Fst between populations:", sparrow_ts.Fst(samples_listed_by_population))

Fst between populations: 0.014022798331082553


### GNN

In [20]:
import pandas as pd

gnn = sparrow_ts.genealogical_nearest_neighbours(
    sparrow_ts.samples(), samples_listed_by_population
)

# Tabulate GNN nicely using a Pandas dataframe with named rows and columns
sample_nodes = [sparrow_ts.node(n) for n in sparrow_ts.samples()]
sample_ids = [n.id for n in sample_nodes]
sample_names = [
    json.loads(sparrow_ts.individual(n.individual).metadata)["name"]
    for n in sample_nodes
]
sample_pops = [
    json.loads(sparrow_ts.population(n.population).metadata)["country"]
    for n in sample_nodes
]
gnn_table = pd.DataFrame(
    data=gnn,
    index=[
        pd.Index(sample_ids, name="Sample node"),
        pd.Index(sample_names, name="Bird"),
        pd.Index(sample_pops, name="Country"),
    ],
    columns=[json.loads(p.metadata)["country"] for p in sparrow_ts.populations()],
)

print(gnn_table)
# Summarize GNN for all birds from the same country
print(gnn_table.groupby(level="Country").mean())

                               Norway    France
Sample node Bird    Country                    
0           8934547 Norway   0.541366  0.458634
1           8934547 Norway   0.546661  0.453339
2           8L19766 Norway   0.515789  0.484211
3           8L19766 Norway   0.483710  0.516290
4           8M31651 Norway   0.534920  0.465080
5           8M31651 Norway   0.595293  0.404707
6           8N05890 Norway   0.555456  0.444544
7           8N05890 Norway   0.542878  0.457122
8           8N73604 Norway   0.548133  0.451867
9           8N73604 Norway   0.508079  0.491921
10          FR041   France   0.422871  0.577129
11          FR041   France   0.471917  0.528083
12          FR044   France   0.477564  0.522436
13          FR044   France   0.407499  0.592501
14          FR046   France   0.520069  0.479931
15          FR046   France   0.498583  0.501417
16          FR048   France   0.479917  0.520083
17          FR048   France   0.467380  0.532620
18          FR050   France   0.556645  0

## Statistics

In [21]:
# サンプルデータの生成

from IPython.display import Markdown
import msprime
import numpy as np

demography = msprime.Demography()
demography.add_population(name="A", initial_size=10_000)
demography.add_population(name="B", initial_size=10_000)
demography.set_symmetric_migration_rate(["A", "B"], 0.001)
ts = msprime.sim_ancestry(
    samples={"A": 2, "B": 2},
    sequence_length=1000,
    demography=demography,
    recombination_rate=2e-8,
    random_seed=12)
ts = msprime.sim_mutations(ts, rate=2e-8, random_seed=12)
Markdown(
    f"These examples use a tree sequence of {ts.num_samples} samples "
    f"in {ts.num_populations} populations, "
    f"with a sequence length of {int(ts.sequence_length)}. "
    f"There are {ts.num_trees} trees and "
    f"{ts.num_sites} variable sites in the tree sequence."
)

These examples use a tree sequence of 8 samples in 2 populations, with a sequence length of 1000. There are 7 trees and 6 variable sites in the tree sequence.

### Basic calling convention

In [22]:
pi = ts.diversity()
print(pi) # Genetic diversity within the sample set

0.001964285714285714


### Restrict to sample sets

In [23]:
pi_0 = ts.diversity(sample_sets=ts.samples(population=0))
print(pi_0)  # Genetic diversity within population 0

0.0013333333333333333


### Summarise in genomic windows

In [24]:
pi_window = ts.diversity(sample_sets=ts.samples(population=1), windows=[0, 400,  600, 1000])
print(pi_window)  # Genetic diversity within population 1 in three windows along the genome

[0.00125 0.0025  0.00375]


### Compare between sample sets

In [25]:
dxy = ts.divergence(sample_sets=[ts.samples(population=0), ts.samples(population=1)])
print(dxy)  # Av number of differences per bp between samples in population 0 and 1

0.002


### Change the mode

In [26]:
bl = ts.divergence(
    mode="branch",  # Use branch lengths rather than genetic differences
    sample_sets=[ts.samples(population=0), ts.samples(population=1)],
)
print(bl)  # Av branch length separating samples in population 0 and 1

75549.0842356655


## Single site statistics

### Mode

In [27]:
ts.diversity(mode="branch")

74860.28301677053

### Sample sets and indexes

In [28]:
ts.diversity(sample_sets=ts.samples(population=0))

0.0013333333333333333

## Multi-way methods

In [29]:
ts.divergence(
    sample_sets=[
        ts.samples(population=0),
        ts.samples(population=1),
    ]
)

0.002

### Windows

In [30]:
num_windows = 4
ts.Tajimas_D(windows=np.linspace(0, ts.sequence_length, num_windows + 1))

array([        nan, -1.31009224,  1.16649684, -0.81245539])

### Span normalise

In [31]:
!pip freeze

absl-py==1.3.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.3.1
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
asciitree==0.3.3
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
atari-py==0.2.9
atomicwrites==1.4.1
attrs==21.4.0
audioread==3.0.0
autograd==1.5
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.9
bokeh==2.3.3
branca==0.6.0
bs4==0.0.1
CacheControl==0.12.11
cachetools==5.2.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.8
colorama==0.4.6
colorcet==3.0.1
coloredlogs==15.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.3
cons==0.4.5
contextlib2==0.5.5
contourpy==1.0.6
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.2
cycler==0.11.0
cymem==2.0.7
Cy